In [1]:
#Связать все с другими файлами!

In [2]:
#Функция для удаления дубликатов
def unique(lst):
    seen = set()
    result = []
    for x in lst:
        if x in seen:
            continue
        seen.add(x)
        result.append(x)
    return result

In [3]:
#вывести список групп
def grList():
    import sqlite3
    #Создание списка из всех направлений...
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT grp_spec_grp_spec_name FROM stud_spisok "
    cursor.execute(sql)
    #print(cursor.fetchall())
    specials=cursor.fetchall()

    conn.commit()
    conn.close()

    #Делаем список из специальностей
    spec=[]
    for i in range(len(unique(specials))):
        spec.append(unique(specials)[i][0])
    return(spec)

In [4]:
grList()

['Экономика',
 'Бухгалтерский учет, анализ и аудит',
 'Финансы и кредит',
 'Коммерция',
 'Менеджмент',
 'Экономическая теория',
 'Национальная экономика',
 'Статистика',
 'Управление персоналом',
 'Мировая экономика',
 'Экономика труда',
 'Экономика (беспрофильный бакалавриат)',
 'Общеэкономическая подготовка',
 'Экономическая безопасность']

In [15]:
#Возьмем оценки за 1 семестр у Экономики по математическому анализу

In [5]:
#Ищем id студентов выбранной специальности
def cod(spec):
    import sqlite3
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT stud_kod FROM stud_spisok WHERE grp_spec_grp_spec_name = ?"
    cursor.execute(sql,[(spec)])
    #print(cursor.fetchall())
    students_kod=cursor.fetchall()

    conn.commit()
    conn.close()

    kod=[]
    for i in range(len(students_kod)):
        kod.append(students_kod[i][0])
    return(kod)

In [5]:
#cod(grList()[0])

In [6]:
slov_summ_ball=dict.fromkeys(cod('Экономика'))

In [7]:
#Находим список предметов, по которым были экзамены у выбранного направления
def predms(ID,semestr):
    import sqlite3
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT predmet_name FROM stud_spisok_ball WHERE stud_kod = ? AND cd_type_name= ? AND semestr= ?"
    cursor.execute(sql,[(ID),('Экзамен'),(semestr)])
    #print(cursor.fetchall())
    students_kod=cursor.fetchall()

    conn.commit()
    conn.close()

    predm=[]
    for i in range(len(unique(students_kod))):
        predm.append(unique(students_kod)[i][0])
    return(predm)

In [15]:
predms(cod('Экономика')[0],4)

['Иностранный язык профессиональный',
 'Бухгалтерский учет',
 'Статистика',
 'Методы оптимальных решений',
 'Макроэкономика']

In [8]:
#Выводим оценки за мат.анализ для каждого id:
#Ищем id студентов выбранной специальности
def markses(ID,predmet):
    import sqlite3
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #matan=u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0439 \u0430\u043d\u0430\u043b\u0438\u0437'
    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT ball FROM stud_spisok_ball WHERE stud_kod = ? AND predmet_name=?"
    cursor.execute(sql,[(ID),(predmet)])
    #print(cursor.fetchall())
    marks=cursor.fetchall()

    conn.commit()
    conn.close()

    new=[]
    for i in marks:
        if i[0]==None: new.append(0)
        else:new.append(i[0])
    return(new) 

In [9]:
#Делаем словарь из баллов и id студента
for i in cod('Экономика'):
    slov_summ_ball[i]=sum(markses(i,'Математический анализ'))

In [10]:
#Ищем, есть ли данный предмет в следующем семестре:
#Возьмем мат анализ:
#Если исходного предмета нет, находим похожий на него предмет.
import numpy as np

same_predmets=[predms(cod("Экономика")[0],1)[2]]
for vt in range(1,8):
    if same_predmets[vt-1] in predms(cod("Экономика")[0],vt+1):
        same_predmets.append(same_predmets[vt-1])
    else:
        tekB=[] #--сюда мы заносим итоговые суммы для каждого предмета
        slov_summ_ball_new=dict.fromkeys(cod('Экономика'))
        for ThisPredm in predms(cod('Экономика')[0],vt+1):
            for i in cod('Экономика'):
                slov_summ_ball_new[i]=sum(markses(i,ThisPredm))
            #следующий семестр
            slov_summ_ball_new_lists = list(slov_summ_ball_new.items())
            slov_summ_ball_new_lists.sort(key=lambda item: item[1])
            #текущий семестр
            slov_summ_ball_lists = list(slov_summ_ball.items())
            slov_summ_ball_lists.sort(key=lambda item: item[1])
            ItogSum=0
            for i in range(len(slov_summ_ball_lists)):
                if slov_summ_ball_lists[i][1]!=0:
                    ItogSum+=abs(i-slov_summ_ball_new_lists.index((slov_summ_ball_lists[i][0],slov_summ_ball_new[slov_summ_ball_lists[i][0]])))
            tekB.append(ItogSum)
        same_predmets.append(predms(cod("Экономика")[0],vt+1)[tekB.index(min(tekB))])
            #print(ItogSum)

In [20]:
slov_summ_ball_new=dict.fromkeys(cod('Экономика'))
for i in cod('Экономика'):
    slov_summ_ball_new[i]=sum(markses(i,'Теория вероятностей и математическая статистика'))

In [21]:
#следующий семестр
slov_summ_ball_new_lists = list(slov_summ_ball_new.items())
slov_summ_ball_new_lists.sort(key=lambda item: item[1])

In [25]:
#текущий семестр
slov_summ_ball_lists = list(slov_summ_ball.items())
slov_summ_ball_lists.sort(key=lambda item: item[1])

In [23]:
import numpy as np
ItogSum=0
for i in range(len(slov_summ_ball_lists)):
    if slov_summ_ball_lists[i][1]!=0:
        ItogSum+=abs(i-slov_summ_ball_new_lists.index((slov_summ_ball_lists[i][0],slov_summ_ball_new[slov_summ_ball_lists[i][0]])))

In [17]:
#same_predmets

In [38]:
import numpy as np


if predms(cod("Экономика")[0],1)[2] in predms(cod("Экономика")[0],2):
    same_predmets.append(same_predmets[vt])
else:
    slov_summ_ball_new=dict.fromkeys(cod('Экономика'))
    for ThisPredm in predms(cod('Экономика')[0],2):
        for i in cod('Экономика'):
            slov_summ_ball_new[i]=sum(markses(i,ThisPredm))
        #следующий семестр
        slov_summ_ball_new_lists = list(slov_summ_ball_new.items())
        slov_summ_ball_new_lists.sort(key=lambda item: item[1])
        #текущий семестр
        slov_summ_ball_lists = list(slov_summ_ball.items())
        slov_summ_ball_lists.sort(key=lambda item: item[1])
        ItogSum=0
        for i in range(len(slov_summ_ball_lists)):
            if slov_summ_ball_lists[i][1]!=0:
                ItogSum+=abs(i-slov_summ_ball_new_lists.index((slov_summ_ball_lists[i][0],slov_summ_ball_new[slov_summ_ball_lists[i][0]])))
        print(ItogSum)

927220
949565
948600
0
844171
